

# Load the Google Drive Helper and Mount

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Install Required Packages

In [ ]:
!pip install huggingface-hub
!huggingface-cli login
!pip install evaluate pycocoevalcap


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `di725-p3-eval` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `di72

In [ ]:
!pip install sacrebleu

# Preprocessing

**Imports & Load Data**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
import wandb

In [ ]:
!python '/content/drive/MyDrive/DI 725 Project/preprocess_p3.py' \
  --input_csv '/content/drive/MyDrive/DI 725 Project/data/captions.csv' \
  --output_csv '/content/drive/MyDrive/DI 725 Project/data/subset_p3_100000_exploded.csv' \
  --seed 42

Saved 178080 train, 22260 validation, 22265 test caption rows to /content/drive/MyDrive/DI 725 Project/data/subset_p3_100000_exploded.csv


In [ ]:
captions_processed_csv = '/content/drive/MyDrive/DI 725 Project/data/subset_p3_100000_exploded.csv'
# Load captions DataFrame
df_p = pd.read_csv(captions_processed_csv)

In [ ]:
df_p

,image,split,caption
0,NWPU_25287.jpg,train,The white snow on the snow-capped mountains is...
1,NWPU_2770.jpg,val,The beach is made up of deep green seawater an...
2,NWPU_3436.jpg,train,A bridge built on a dark green river and the f...
3,NWPU_16524.jpg,train,Here is a road with mobile homes on both sides .
4,NWPU_7352.jpg,val,A dense residential area with lots of trees an...
...,...,...,...
222600,UCM_997.jpg,train,Green turfs and some trees in the golf course .
222601,NWPU_1999.jpg,train,Many buildings are next to the basketball court .
222602,NWPU_30881.jpg,train,Two yellow airplanes with a red pattern were...
222603,NWPU_5270.jpg,train,Three neatly arranged circular green fields .


# Running the Code for Fine-tuning with LoRA

In [ ]:
!python "/content/drive/MyDrive/DI 725 Project/train_lora_p3_final.py" \ # the training script
  --csv   '/content/drive/MyDrive/DI 725 Project/data/subset_p3_100000_exploded.csv' \
  --images_dir "/content/drive/MyDrive/DI 725 Project/data/resized" \
  --output_dir '/content/drive/MyDrive/DI 725 Project/out/lora_r4_p3_final' \ # change it according to the selected model
  --wandb_project "di725-project" \
  --run_name "phase3-lora-r4-final" \ # change it according to the selected experiment
  --fp16 \ # turn on mixed-precision training - 16-bit floats
  --max_samples 100000 \ # limit training samples to 100000
  --train_bs 4 \ # training batch size - kept constant for all experiments
  --eval_bs 4 \ # evaluation batch size - kept constant for all experiments
  --epochs 1 \ # number of epochs - kept constant for all experiments
  --lr 1e-6 \ # learning rate
  --lora_r 4 \ # lora rank - adjusted during experiments
  --lora_alpha 16 \ # lora alpha - adjusted during experiments
  --lora_dropout 0.1 \ # lora dropout - adjusted during experiments
  --gradient_accumulation_steps 4 # kept constant for all experiments

Görüntülenen çıkış son 5000 satıra kısaltıldı.
 80% 5005/6250 [1:12:06<33:10,  1.60s/it]You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your te

# Base Model PaliGemma Inference

In [ ]:
!python "/content/drive/MyDrive/DI 725 Project/evaluate_p3_final.py" \
  --model_dir "google/paligemma-3b-pt-224" \
  --processor_name "google/paligemma-3b-pt-224" \
  --csv      "/content/drive/MyDrive/DI 725 Project/data/subset_p3_100000_exploded.csv" \
  --images_dir    "/content/drive/MyDrive/DI 725 Project/data/resized" \
  --split         test \
  --batch_size    8 \
  --device cuda \
  --max_length    64 \
  --num_beams 4 \
  --wandb_project di725-project \
  --run_name      phase3-eval-baseline


2025-06-16 18:44:25.109024: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 18:44:25.126934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750099465.149306   33184 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750099465.155773   33184 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-16 18:44:25.177814: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Evaluation of LoRA

**Example For Rank 4**




In [ ]:
!python "/content/drive/MyDrive/DI 725 Project/evaluate_p3_final.py" \
  --model_dir '/content/drive/MyDrive/DI 725 Project/out/lora_r4_p3_final' \ # adjust according to model path
  --processor_name "google/paligemma-3b-pt-224" \
  --csv "/content/drive/MyDrive/DI 725 Project/data/subset_p3_100000_exploded.csv" \
  --images_dir "/content/drive/MyDrive/DI 725 Project/data/resized" \
  --split test \
  --batch_size 8 \ # kept constant during experiments
  --device cuda \
  --max_length 64 \ # kept constant during experiments
  --num_beams 4 \ # kept constant during experiments
  --wandb_project di725-project \
  --run_name eval-lora-r4-final # adjust according to selected model

2025-06-16 19:10:18.077550: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 19:10:18.094524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750101018.115664   39696 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750101018.122167   39696 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-16 19:10:18.143013: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr